# Similarity of CCL28 fragment hits to other molecules in the fragment screens using Tanimoto fingerprints

In [ ]:
import pandas as pd
from rdkit.Chem import AllChem

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

from rdkit import Chem, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import Draw

# All we need for clustering
from scipy.cluster.hierarchy import dendrogram, linkage

# First I will read  in an excel file with all of the hits from CCL28, CCL20 and CXCL12 fragment screens
# One of the columnes, labeled smiles, is the smiles string for each fragment in the list.

In [ ]:

hits = pd.read_excel('/Users/djensen/Documents/CCL28/CCL28_frag_screen_hits.xlsx', skiprows=2)
hits

## The next section is heavily borrowed and influenced from examples on this page


https://chem-workflows.com/articles/2019/06/28/similarity-analysis-of-compound-databases/

# Create an empty list of molecules, we will append to them as we read through the list of hits.

In [ ]:
hit_list = []

# Read through the smiles and generate RDkit molecules, append them to the hit_list

In [ ]:
counter = 0
for item in hits.smiles:
    mol=Chem.MolFromSmiles(item) # Converting SMILES codes into rdkit mol 
    mol.SetProp('_Name', str(hits.Cdid[counter])) # Adding the name for each molecule
    hit_list.append(mol)
    counter = counter + 1

In [ ]:
Draw.MolsToGridImage(hit_list,molsPerRow=6,subImgSize=(250,250),legends=[mol.GetProp('_Name') for mol in hit_list])


In [ ]:
sl = [hit_list[11], hit_list[12]]
sl2 = [hit_list[5], hit_list[9]]
sl3 = [hit_list[16], hit_list[3], hit_list[6]]
sl4 = [hit_list[7], hit_list[10], hit_list[4]]

In [ ]:
Draw.MolsToGridImage(sl,molsPerRow=6,subImgSize=(250,250),legends=[mol.GetProp('_Name') for mol in sl])

In [ ]:
Draw.MolsToGridImage(sl2,molsPerRow=6,subImgSize=(250,250),legends=[mol.GetProp('_Name') for mol in sl2])

In [ ]:
Draw.MolsToGridImage(sl3,molsPerRow=6,subImgSize=(250,250),legends=[mol.GetProp('_Name') for mol in sl3])

In [ ]:
Draw.MolsToGridImage(sl4,molsPerRow=6,subImgSize=(250,250),legends=[mol.GetProp('_Name') for mol in sl4])

# From all the the molecules, generate a "hashed fingerprint" for each molecule
# This is the heavy atom connections that make up the "fingerprint" of each molecule.

In [ ]:
fps = [FingerprintMols.FingerprintMol(mol) for mol in hit_list]

In [ ]:
print(len(hit_list))
print(len(fps))

# create the dendogramwith Tanimoto Similarity as the distance metric

In [ ]:
size=len(hit_list)
hmap=np.empty(shape=(size,size))
table=pd.DataFrame()
for index, i in enumerate(fps):
    for jndex, j in enumerate(fps):
        similarity=DataStructs.FingerprintSimilarity(i,j, metric=DataStructs.TanimotoSimilarity)
        #similarity=DataStructs.FingerprintSimilarity(i,j, metric=DataStructs.DiceSimilarity)
        #similarity=DataStructs.FingerprintSimilarity(i,j, metric=DataStructs.CosineSimilarity)
        #similarity=DataStructs.FingerprintSimilarity(i,j, metric=DataStructs.SokalSimilarity)

        hmap[index,jndex]=similarity
        table.loc[hit_list[index].GetProp('_Name'),hit_list[jndex].GetProp('_Name')]=similarity

In [ ]:
linked = linkage(hmap,'single')
labelList = [mol.GetProp('_Name') for mol in hit_list]

In [ ]:
plt.figure(figsize=(8,15))

ax1=plt.subplot()
o=dendrogram(linked,  
            orientation='left',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)

ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
plt.title('Similarity clustering',fontsize=20,weight='bold')
plt.tick_params ('both',width=2,labelsize=8)
plt.tight_layout()
plt.show() 

In [ ]:
# This will give us the clusters in order as the last plot
new_data=list(reversed(o['ivl']))

# we create a new table with the order of HCL
hmap_2=np.empty(shape=(size,size))
for index,i in enumerate(new_data):
    for jndex,j in enumerate(new_data):
        hmap_2[index,jndex]=table.loc[i].at[j]

In [ ]:
figure= plt.figure(figsize=(30,30))
gs1 = gridspec.GridSpec(2,7)
gs1.update(wspace=0.01)
ax1 = plt.subplot(gs1[0:-1, :2])
dendrogram(linked, orientation='left', distance_sort='descending',show_leaf_counts=True,no_labels=True)
ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = plt.subplot(gs1[0:-1,2:6])
#f=ax2.imshow (hmap_2, cmap='jet', interpolation='nearest')
f=ax2.imshow (hmap_2, cmap='PRGn_r')

ax2.set_title('Fingerprint Similarity of Chemokine fragment Hits',fontsize=30,weight='bold')
ax2.set_xticks (range(len(new_data)))
ax2.set_yticks (range(len(new_data)))
ax2.set_xticklabels (new_data,rotation=90,size=8)
ax2.set_yticklabels (new_data,size=8)

ax3 = plt.subplot(gs1[0:-1,6:7])
m=plt.colorbar(f,cax=ax3,shrink=0.75,orientation='vertical',spacing='uniform',pad=0.01)
m.set_label ('Fingerprint Similarity')

plt.tick_params ('both',width=2)
plt.plot()
plt.savefig('./CCL28_similarity.png', dpi = 600)

# Searching Screening libraries for similar fragments, starting with the fragment below

In [ ]:
hit_list[19]

# Read in all the purchased libraries and create lists of molecules

In [ ]:
suppl_may = Chem.SDMolSupplier('/Users/djensen/Documents/RDKitDJ/New_library_comparison/Library_sdf_files_and_quotes/Purchased_libraries/Maybridge_1000_20181023.sdf') # read mols from SDfile
fps_may = [AllChem.GetMorganFingerprint(m,2) for m in suppl_may] # calculate morgan fingerprints for all compounds in SDfile

suppl_ena = Chem.SDMolSupplier('/Users/djensen/Documents/RDKitDJ/New_library_comparison/Library_sdf_files_and_quotes/Purchased_libraries/Enamine_1080_Multiplexed_20200310.sdf') # read mols from SDfile
fps_ena = [AllChem.GetMorganFingerprint(m,2) for m in suppl_ena] # calculate morgan fingerprints for all compounds in SDfile

suppl_tar = Chem.SDMolSupplier('/Users/djensen/Documents/RDKitDJ/New_library_comparison/Library_sdf_files_and_quotes/Purchased_libraries/TargetMol_246_Multiplexed_20200304.sdf') # read mols from SDfile
fps_tar = [AllChem.GetMorganFingerprint(m,2) for m in suppl_tar] # calculate morgan fingerprints for all compounds in SDfile


# Search a single library for similar molecule, specifying a single library and molecule with a threshold

In [ ]:
def similarity_search(cdi = None, smi=None, fps = None, suppl = None, thresh=0.9):
    
    '''
    This function returns all compounds in library with tanimoto coefficients above the 
    threshold. Either a library compound ID (cid=) or smiles pattern (smi=) must be 
    passed. If no tanimoto coefficient threshold (thresh=) is passed, 0.9 will be used.
    '''
    m = Chem.MolFromSmiles(smi)
    m.SetProp('_Name', cdi)
    fp = AllChem.GetMorganFingerprint(m, 2)
    # search for mols with similarity above threshold
    mols = [m]
    for ix, m in enumerate(suppl):
        similarity = DataStructs.DiceSimilarity(fp, fps[ix])
        if similarity >= thresh:
            AllChem.Compute2DCoords(m)
            mols.append(m)

    # show similar compounds
    names = [m.GetProp('_Name') for m in mols]
    img = Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(200, 200), legends=names)
    
    return img

In [ ]:
smiles = 'OC(=O)c1cc2ccccc2[nH]1'
compound_id = '1003'
img = similarity_search(cdi = compound_id, smi = smiles, fps = fps_tar, suppl=suppl_tar, thresh=0.6)
#img = similarity_search(smi=smiles, thresh=0.5)
#img.save('similarity_search.png')
img

In [ ]:
smiles = 'OC(=O)c1cc2ccccc2[nH]1'
compound_id = '1003'
img = similarity_search(cdi = compound_id, smi = smiles, fps = fps_may, suppl = suppl_may, thresh=0.6)
#img = similarity_search(smi=smiles, thresh=0.5)
#img.save('similarity_search.png')
img

In [ ]:
smiles = 'OC(=O)c1cc2ccccc2[nH]1'
compound_id = '1003'
img = similarity_search(cdi = compound_id, smi = smiles, fps = fps_ena, suppl=suppl_ena, thresh=0.6)
#img = similarity_search(smi=smiles, thresh=0.5)
#img.save('similarity_search.png')
img

# Search a list of libraries for similar molecule, specifying a single library and molecule with a threshold.
# Output is a list of similar molecules and a plot of the similar molecules

In [ ]:
def all_similarity_search(cdi = None, smi=None, fps = None, suppl = None, thresh=0.9):
    
    '''
    This function returns all compounds in library with tanimoto coefficients above the 
    threshold. Either a library compound ID (cid=) or smiles pattern (smi=) must be 
    passed. If no tanimoto coefficient threshold (thresh=) is passed, 0.9 will be used.
    '''
    m = Chem.MolFromSmiles(smi)
    m.SetProp('_Name', cdi)
    fp = AllChem.GetMorganFingerprint(m, 2)
    # search for mols with similarity above threshold
    mols = [m]
    for ind_suppl in suppl:
        for ix, m in enumerate(ind_suppl):
            similarity = DataStructs.DiceSimilarity(fp, fps[ix])
            if similarity >= thresh:
                AllChem.Compute2DCoords(m)
                mols.append(m)

    # show similar compounds
    names = [m.GetProp('_Name') for m in mols]
    img = Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(200, 200), legends=names)
    
    return mols, img

In [ ]:
smiles = 'OC(=O)c1cc2ccccc2[nH]1'
compound_id = '1003'
sim_mols, sim_img = all_similarity_search(cdi = compound_id, smi = smiles, fps = fps_ena, suppl=[suppl_ena, suppl_may, suppl_tar], thresh=0.55)


In [ ]:
sim_img

In [ ]:
sim_mols[0]

In [ ]:
sim_mols[1]

In [ ]:
sim_mols[2]

In [ ]:
sim_mols[0]